### CSV 파일

In [1]:
import numpy as np
import pandas as pd
import torch

# ✅ 입력 CSV 파일 경로 설정
input_path = "FA_entities (1).csv"

# ✅ CSV 파일 불러오기 및 전처리
df1 = pd.read_csv(input_path)

# ✅ entity_text 열에서 유효한 문장 수 확인
num_entities = df1['entity_text'].dropna().shape[0]
print(f"✅ 불러온 entity_text 문장 수: {num_entities}")

✅ 불러온 entity_text 문장 수: 7406


### 핵심 파라미터 설정

In [2]:
# ✅ HDBSCAN 클러스터링 기준

HDBSCAN_MIN_CLUSTER_SIZE = 2
# 하나의 클러스터가 되기 위해 포함해야 할 최소 문장 수입니다.
# 이보다 작은 연결 집합은 클러스터로 간주되지 않고 노이즈로 처리됩니다.
# 값이 너무 크면 소규모 의미 그룹이 무시되고, 너무 작으면 품질이 낮은 군집이 생성될 수 있습니다.

HDBSCAN_MIN_SAMPLES = 2
# 핵심 포인트(core point)로 간주되기 위해 요구되는 최소한의 이웃 수입니다.
# 이 값만큼의 문장이 일정 거리 이내에 있어야 해당 문장은 고밀도 지역으로 판단됩니다.
# 값이 클수록 더 밀집된 클러스터만 생성되며, 작을수록 느슨한 클러스터도 허용됩니다.

In [3]:
## ✅ 아래의 값들은 자동으로 처리됩니다. 필요 시 수동으로 처리하세요.

# TOP_K_RATIO = 0.2
# # 전체 문장 중에서 각 문장이 검색할 Top-K 이웃의 비율입니다.
# # 예를 들어 문장이 7400개이고 0.1이라면, 각 문장은 의미적으로 가장 유사한 740개의 이웃을 찾습니다.
# # 이 Top-K는 의미적 분포(semantic neighborhood)를 형성하기 위함입니다.

# TAU_LOCAL = 0.8
# # Top-K 이웃 중에서도 얼마나 유사한 이웃을 "진짜 이웃"으로 간주할지 결정하는 임계값입니다.

# TAU_ASSIGN = 0.4
# # 대표 문장을 기준으로 다른 문장을 할당(매핑)할 때 적용할 최소 유사도 기준입니다.
# # 즉, 코사인 유사도가 설정값 이상인 경우에만 해당 문장을 해당 대표 문장에 소속시킵니다.
# # 군집화 이후, 비할당 문장(noise 포함)에 대해 재할당할 때 사용됩니다.

### 라이브러리 버전 확인 및 모델 로드

In [4]:
# conda install -c pytorch faiss-gpu

In [5]:
import importlib
import pkg_resources

# pip 기반 확인 패키지 목록 (faiss-gpu는 제외)
pip_packages = [
    "sentence-transformers",
    "transformers",
    "torch",
    "numpy",
    "scikit-learn",
    "scipy",
    "pandas",
    "tqdm"
]

print("📦 현재 설치된 라이브러리 버전:\n")

# pip 기반 패키지 확인
for pkg in pip_packages:
    try:
        version = pkg_resources.get_distribution(pkg).version
        print(f"{pkg:20} ✅ {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{pkg:20} ❌ 설치되지 않음")

# faiss-gpu는 직접 import로 확인
print("\n📦 faiss-gpu 상태 확인:")
try:
    import faiss
    ngpus = faiss.get_num_gpus() if hasattr(faiss, "get_num_gpus") else "unknown"
    print(f"faiss-gpu           ✅ import 성공 (GPU 사용 가능: {ngpus})")
except ImportError:
    print("faiss-gpu           ❌ 설치되지 않음 (import 실패)")


📦 현재 설치된 라이브러리 버전:

sentence-transformers ✅ 2.6.1
transformers         ✅ 4.49.0
torch                ✅ 2.6.0
numpy                ✅ 1.26.4
scikit-learn         ✅ 1.5.0
scipy                ✅ 1.13.0
pandas               ✅ 2.2.2
tqdm                 ✅ 4.65.0

📦 faiss-gpu 상태 확인:
faiss-gpu           ✅ import 성공 (GPU 사용 가능: 1)


/tmp/ipykernel_22387/3511165994.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [6]:
from sentence_transformers import SentenceTransformer
from huggingface_hub import snapshot_download
import os

# ✅ 사용자 지정 모델 저장 경로
local_model_dir = "/data/models/all-MiniLM-L6-v2" 

##################### 모델 저장 경로를 원하는 곳으로 변경하세요.

# ✅ 모델을 해당 경로로 다운로드 (이미 있으면 다운로드하지 않음)
snapshot_download(
    repo_id="sentence-transformers/all-MiniLM-L6-v2",
    local_dir=local_model_dir,
    local_dir_use_symlinks=False
)

# ✅ 로컬 경로에서 모델 로드
model = SentenceTransformer(local_model_dir)

/miniconda/lib/python3.12/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

### 프로세스 진행

In [7]:
# ✅ 문장 리스트 추출
sentences = df1['entity_text'].dropna().astype(str).tolist()

# ✅ 문장 임베딩 (정규화 포함)
embeddings = model.encode(sentences, normalize_embeddings=True)
embeddings = np.array(embeddings)

print(f"✅ 임베딩 완료. Shape: {embeddings.shape}")

✅ 임베딩 완료. Shape: (7406, 384)


In [8]:
import faiss

# ✅ TOP_K_RATIO 자동 설정 (사전 정의되어 있지 않으면 평균 유사도 기반 휴리스틱 사용)
try:
    TOP_K_RATIO
except NameError:
    TOP_K_RATIO = None

if TOP_K_RATIO is None:
    # ✅ Faiss 인덱스 생성 (코사인 유사도용 내적 기반)
    index_tmp = faiss.IndexFlatIP(embeddings.shape[1])
    index_tmp.add(embeddings)

    # ✅ 각 문장마다 가장 유사한 Top-100 문장을 검색
    # - 자기 자신 포함: 유사도 1.0이므로 제외 예정
    # - 100은 실용적인 균형값 (너무 적으면 분포 불안정, 너무 많으면 계산 비용 증가)
    D_tmp, _ = index_tmp.search(embeddings, min(100, len(embeddings)))

    # ✅ 자기 자신 제외한 유사도만 사용 (즉, 상위 99개의 유사 문장)
    sim_scores = D_tmp[:, 1:]

    # ✅ 평균 유사도를 기반으로 TOP_K_RATIO 계산
    # - 예: 평균 유사도가 0.73이면 전체 문장 중 73%를 의미 이웃으로 고려
    avg_sim = np.mean(sim_scores)

    # ✅ 안정성을 위해 상한/하한 제한 (과도한 군집 또는 의미 없는 분산 방지)
    avg_ratio = max(min(avg_sim, 0.95), 0.05)

    # ✅ 반올림 처리 후 최종 설정
    TOP_K_RATIO = round(avg_ratio, 2)

    print(f"✅ 평균 유사도 기반 TOP_K_RATIO 자동 설정: {TOP_K_RATIO}")

# ✅ Top-K 계산 (전체 문장에서 설정된 비율만큼 이웃을 고려)
TOP_K = int(len(embeddings) * TOP_K_RATIO)

# ✅ 본 인덱스 생성 및 유사 문장 검색
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)
D, I = index.search(embeddings, TOP_K)

print(f"✅ TOP_K_RATIO 설정됨: {TOP_K_RATIO} (K = {TOP_K})")
print(f"✅ 유사 문장 검색 완료: shape = {I.shape}")

✅ 평균 유사도 기반 TOP_K_RATIO 자동 설정: 0.5400000214576721
✅ TOP_K_RATIO 설정됨: 0.5400000214576721 (K = 3999)
✅ 유사 문장 검색 완료: shape = (7406, 3999)


In [9]:
# ✅ 문장 단위 군집 목록 초기화

# 🔧 TAU_LOCAL이 미리 정의되지 않았다면 자동 설정
try:
    TAU_LOCAL
except NameError:
    TAU_LOCAL = None

if TAU_LOCAL is None:
    # ✅ 평균 유사도 기반 자동 설정
    # - 자기 자신을 제외한 Top-K 유사도 값만 사용
    all_sim = D[:, 1:].flatten()
    avg_sim = np.mean(all_sim)

    # ✅ 평균이 0.7 이상이면 그대로 사용, 아니면 최소 0.7로 고정
    TAU_LOCAL = round(max(avg_sim, 0.7), 4)

    print(f"✅ 평균 기반 TAU_LOCAL 자동 설정: {TAU_LOCAL}")

# ✅ 문장 단위 의미 군집 생성
local_clusters = []
for idx, neighbors in enumerate(I):
    cluster = [i for i, score in zip(neighbors, D[idx]) if score >= TAU_LOCAL and i != idx]
    if cluster:
        cluster.append(idx)  # 자기 자신 포함
        local_clusters.append(set(cluster))

print(f"✅ 의미 군집 수: {len(local_clusters)}")

✅ 평균 기반 TAU_LOCAL 자동 설정: 0.7
✅ 의미 군집 수: 5994


In [10]:
import hdbscan
from collections import defaultdict

# ✅ HDBSCAN 클러스터러 설정 및 학습
# - min_cluster_size: 하나의 클러스터가 최소로 포함해야 할 문장 수
# - min_samples: 각 문장이 "핵심 포인트"가 되기 위한 최소 이웃 수
# - metric='euclidean': 거리 계산 기준 (보통 문장 임베딩은 유클리드 또는 코사인)
# - cluster_selection_method='eom': 클러스터 선택 기준 (EOM이 기본)

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
    min_samples=HDBSCAN_MIN_SAMPLES,
    metric="euclidean",   
    cluster_selection_method="eom"
)

# ✅ 클러스터링 수행 → 각 문장에 대해 클러스터 라벨 예측
cluster_labels = clusterer.fit_predict(embeddings)

# ✅ 클러스터 라벨을 기준으로 문장 인덱스를 그룹핑
# - noise(-1)는 제외
label_to_indices = defaultdict(list)
for idx, label in enumerate(cluster_labels):
    if label != -1:  # -1은 클러스터로 간주되지 않는 노이즈
        label_to_indices[label].append(idx)

# ✅ 각 클러스터를 하나의 문장 인덱스 집합으로 구성
merged_clusters = [set(indices) for indices in label_to_indices.values()]

print(f"✅ HDBSCAN 기반 병합 후 군집 수: {len(merged_clusters)}")

✅ HDBSCAN 기반 병합 후 군집 수: 941


In [11]:
#### GPU기반 HDBSCAN 이용이 해당 코드를 사용하세요.

# from cuml.cluster import HDBSCAN  # ✅ cuML의 GPU 버전 HDBSCAN
# from collections import defaultdict

# # ✅ HDBSCAN 클러스터러 설정 및 학습 (GPU 기반)
# # - min_cluster_size: 하나의 클러스터가 최소로 포함해야 할 문장 수
# # - min_samples: 각 문장이 "핵심 포인트"가 되기 위한 최소 이웃 수
# # - metric='euclidean': 거리 계산 기준 (cuML HDBSCAN은 'euclidean'만 지원)
# # - cluster_selection_method='eom': 클러스터 선택 기준 (EOM이 기본)

# clusterer = HDBSCAN(
#     min_cluster_size=HDBSCAN_MIN_CLUSTER_SIZE,
#     min_samples=HDBSCAN_MIN_SAMPLES,
#     metric='euclidean',  # cuML은 'euclidean'만 지원
#     cluster_selection_method='eom'
# )

# # ✅ 클러스터링 수행 → 각 문장에 대해 클러스터 라벨 예측
# # - cuML의 fit_predict는 GPU 배열을 반환하므로 numpy로 변환 필요
# cluster_labels = clusterer.fit_predict(embeddings).to_numpy()

# # ✅ 클러스터 라벨을 기준으로 문장 인덱스를 그룹핑
# # - noise(-1)는 제외
# label_to_indices = defaultdict(list)
# for idx, label in enumerate(cluster_labels):
#     if label != -1:  # -1은 클러스터로 간주되지 않는 노이즈
#         label_to_indices[label].append(idx)

# # ✅ 각 클러스터를 하나의 문장 인덱스 집합으로 구성
# merged_clusters = [set(indices) for indices in label_to_indices.values()]

# print(f"✅ HDBSCAN 기반 병합 후 군집 수: {len(merged_clusters)}")

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
from collections import defaultdict

# ✅ 대표 문장 저장용 딕셔너리
# - key: 대표 문장 인덱스
# - value: 해당 클러스터에 포함된 문장 인덱스 목록
cluster_representatives = {}

# ✅ TF-IDF 벡터라이저 준비
# - 단어 기반으로 문장을 벡터화 (stop_words="english" 제거)
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(sentences)

# ✅ 각 클러스터마다 대표 문장 선정
for cluster in merged_clusters:
    cluster_indices = list(cluster)  # 해당 클러스터에 속한 문장 인덱스들
    vectors = embeddings[cluster_indices]  # 문장 임베딩 추출

    # ✅ 의미 중심성 계산
    # - 클러스터 벡터들의 평균(centroid)과 각 문장의 내적 → 중심성 점수
    centroid = np.mean(vectors, axis=0)
    sim_scores = np.dot(vectors, centroid)  # 각 문장이 중심과 얼마나 유사한지

    # ✅ TF-IDF 중요도 계산
    # - 각 문장의 전체 단어의 TF-IDF 합산 (벡터 합계 기준)
    tfidf_scores = tfidf_matrix[cluster_indices].sum(axis=1).A1  # sparse → numpy array

    # ✅ 하이브리드 점수 = 의미 중심성 * TF-IDF
    hybrid_scores = sim_scores * tfidf_scores

    # ✅ 가장 점수가 높은 문장을 대표로 선정
    best_idx = cluster_indices[np.argmax(hybrid_scores)]

    # ✅ 대표 문장 인덱스 → 해당 클러스터 문장 인덱스 저장
    cluster_representatives[best_idx] = cluster_indices

print(f"✅ 하이브리드 방식으로 대표 문장 수: {len(cluster_representatives)}")

# ✅ 대표 문장 간 유사도 기반 병합
# - 유사도 기준: 0.7 이상이면 동일 의미로 판단 → 병합 수행

rep_indices = list(cluster_representatives.keys())
rep_vectors = embeddings[rep_indices]

# ✅ 정규화 (코사인 유사도용)
norms = np.linalg.norm(rep_vectors, axis=1, keepdims=True)
norms[norms == 0] = 1e-8  # 0 나눔 방지
rep_vectors = rep_vectors / norms

# ✅ Faiss 인덱스를 이용한 빠른 유사도 탐색
index = faiss.IndexFlatIP(rep_vectors.shape[1])
index.add(rep_vectors)
D_rep, I_rep = index.search(rep_vectors, len(rep_indices))  # 자기 자신 포함

# ✅ Faiss 내부 인덱스 → 실제 대표 문장 인덱스로 변환
faiss_id_to_rep_idx = {i: rep_indices[i] for i in range(len(rep_indices))}

# ✅ 대표 간 병합 수행
group_map = {}  # 병합 대상 그룹화: 대표 인덱스 → 대표 인덱스 그룹 ID
group_id_counter = 0

for i, sims in enumerate(D_rep):
    rep_i = faiss_id_to_rep_idx[i]
    for j, sim in zip(I_rep[i], sims):
        rep_j = faiss_id_to_rep_idx[j]
        if rep_i == rep_j or sim < 0.7:
            continue  # 자기 자신 또는 유사도 기준 미달은 제외

        # 그룹 할당
        gid_i = group_map.get(rep_i)
        gid_j = group_map.get(rep_j)

        if gid_i is None and gid_j is None:
            group_map[rep_i] = group_id_counter
            group_map[rep_j] = group_id_counter
            group_id_counter += 1
        elif gid_i is not None and gid_j is None:
            group_map[rep_j] = gid_i
        elif gid_j is not None and gid_i is None:
            group_map[rep_i] = gid_j
        elif gid_i != gid_j:
            # 그룹 병합 (작은 그룹 ID로 통일)
            min_gid = min(gid_i, gid_j)
            max_gid = max(gid_i, gid_j)
            for k, v in list(group_map.items()):
                if v == max_gid:
                    group_map[k] = min_gid

# ✅ 병합 그룹 ID → 대표 문장 인덱스 매핑
# - 하나의 그룹을 대표할 대표 문장 인덱스를 다시 선택 (작은 인덱스 우선)
group_id_to_rep_idx = {}
for rep_idx, gid in group_map.items():
    if gid not in group_id_to_rep_idx or rep_idx < group_id_to_rep_idx[gid]:
        group_id_to_rep_idx[gid] = rep_idx

# ✅ 병합된 클러스터 재정의
merged_representatives = defaultdict(list)
for rep_idx, members in cluster_representatives.items():
    group = group_map.get(rep_idx, None)
    if group is None:
        # 병합 대상 아님 → 독립 클러스터 유지
        merged_representatives[rep_idx].extend(members)
    else:
        # 병합된 그룹의 대표 문장 인덱스를 키로 사용
        new_rep_idx = group_id_to_rep_idx[group]
        merged_representatives[new_rep_idx].extend(members)

# ✅ 최종 대표 문장 → 클러스터 구성
final_cluster_representatives = {}
for rep_idx, member_list in merged_representatives.items():
    # 가장 중심성 높은 문장을 대표로 다시 선정
    cluster_indices = list(set(member_list))
    vectors = embeddings[cluster_indices]
    centroid = np.mean(vectors, axis=0)
    sim_scores = np.dot(vectors, centroid)
    best_idx = cluster_indices[np.argmax(sim_scores)]
    final_cluster_representatives[best_idx] = cluster_indices

print(f"✅ 대표 간 병합 후 최종 대표 문장 수: {len(final_cluster_representatives)}")

✅ 하이브리드 방식으로 대표 문장 수: 941
✅ 대표 간 병합 후 최종 대표 문장 수: 615


In [13]:
# ✅ 대표 문장 인덱스 및 임베딩 벡터 추출
cluster_representatives = final_cluster_representatives  # 🔄 병합된 결과를 기반으로 갱신
rep_indices = list(cluster_representatives.keys())
rep_vectors = embeddings[rep_indices]

# ✅ Faiss 인덱스 생성 (코사인 유사도 기반)
# - 코사인 유사도 계산을 위해 L2 정규화 후 내적(IndexFlatIP) 사용
rep_vectors = rep_vectors / np.linalg.norm(rep_vectors, axis=1, keepdims=True)
all_vectors = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

index_q = faiss.IndexFlatIP(rep_vectors.shape[1])  # IP: Inner Product (코사인 유사도)
index_q.add(rep_vectors)

# ✅ 전체 문장에 대해 가장 유사한 대표 문장 1개씩 검색
D_q, I_q = index_q.search(all_vectors, 1)  # D_q: 유사도, I_q: 대표 인덱스 인덱스

# ✅ 노이즈 문장만 유사한 대표 문장에 할당 (기존 클러스터는 유지)
# 📌 기존 클러스터에 이미 포함된 문장 인덱스 수집
already_clustered = set()
for cluster in cluster_representatives.values():
    already_clustered.update(cluster)

# 🔧 TAU_ASSIGN 자동 설정: 대표문장과 클러스터 멤버의 평균 유사도 기반
try:
    TAU_ASSIGN
except NameError:
    TAU_ASSIGN = None

if TAU_ASSIGN is None:
    tau_assign_dict = {}
    for rep_id, cluster in cluster_representatives.items():
        rep_vec = embeddings[rep_id]
        member_vecs = embeddings[cluster]
        member_vecs = member_vecs / np.linalg.norm(member_vecs, axis=1, keepdims=True)
        rep_vec_norm = rep_vec / np.linalg.norm(rep_vec)
        sims = np.dot(member_vecs, rep_vec_norm)
        tau_assign_dict[rep_id] = np.mean(sims)

    print("✅ 대표 문장별 평균 유사도 기반 TAU_ASSIGN 자동 설정 완료")

    # ✅ 대표 문장별 TAU_ASSIGN 값 출력 (상위 10개만 예시)
    print("\n📊 대표 문장별 자동 설정 TAU_ASSIGN (평균 유사도 기준):")
    for i, (rep_id, tau) in enumerate(tau_assign_dict.items()):
        print(f"- 대표 문장 idx={rep_id} | TAU_ASSIGN={round(tau, 4)} | \"{sentences[rep_id][:70]}{'...' if len(sentences[rep_id]) > 70 else ''}\"")
        if i >= 9:
            print("... (생략: 총 {}개 대표 문장)".format(len(tau_assign_dict)))
            break
else:
    tau_assign_dict = {rep_id: TAU_ASSIGN for rep_id in rep_indices}

# ✅ 새롭게 할당된 노이즈 문장 저장소
noise_reassignments = {}

# 🔁 전체 문장 중에서 노이즈 문장만 유사도 기반으로 할당
for i, (sim, q_idx) in enumerate(zip(D_q[:, 0], I_q[:, 0])):
    if i in already_clustered:
        continue  # 이미 HDBSCAN 클러스터에 속한 문장은 건너뜀

    rep_id = rep_indices[q_idx]  # 대표 문장 인덱스
    if sim >= tau_assign_dict[rep_id]:
        noise_reassignments.setdefault(rep_id, []).append(i)

# ✅ 기존 cluster_representatives에 노이즈 문장 추가 (병합)
for rep_id, new_indices in noise_reassignments.items():
    cluster_representatives[rep_id].extend(new_indices)

# ✅ 최종 결과 출력
print(f"✅ 재할당된 노이즈 문장 수: {sum(len(v) for v in noise_reassignments.values())}")
print(f"✅ 최종 문장 할당이 완료된 대표 문장 수: {len(cluster_representatives)}")

✅ 대표 문장별 평균 유사도 기반 TAU_ASSIGN 자동 설정 완료

📊 대표 문장별 자동 설정 TAU_ASSIGN (평균 유사도 기준):
- 대표 문장 idx=78 | TAU_ASSIGN=0.882099986076355 | "health, safety, and/or security related event"
- 대표 문장 idx=3 | TAU_ASSIGN=0.8884999752044678 | "higher risk of being alone and experiencing an emergency"
- 대표 문장 idx=4337 | TAU_ASSIGN=0.6288999915122986 | "elderly patients"
- 대표 문장 idx=4 | TAU_ASSIGN=0.9336000084877014 | "current solutions to these problems are not sufficient"
- 대표 문장 idx=3753 | TAU_ASSIGN=0.7605000138282776 | "personal emergency response system"
- 대표 문장 idx=52 | TAU_ASSIGN=1.0 | "gesture recognizer module"
- 대표 문장 idx=7 | TAU_ASSIGN=1.0 | "emergency event module"
- 대표 문장 idx=32 | TAU_ASSIGN=1.0 | "proximity detection module"
- 대표 문장 idx=33 | TAU_ASSIGN=1.0 | "notification module"
- 대표 문장 idx=5125 | TAU_ASSIGN=0.9133999943733215 | "individuals"
... (생략: 총 615개 대표 문장)
✅ 재할당된 노이즈 문장 수: 149
✅ 최종 문장 할당이 완료된 대표 문장 수: 615


In [14]:
import json
from collections import Counter
from IPython.display import display, HTML

# ✅ 전체 문장 수
total_sentences = len(sentences)

# ✅ 대표 문장 수
num_representatives = len(cluster_representatives)

# ✅ 대표 문장별 할당 문장 수 계산
data = []
all_assigned_texts = []

for rep_idx, indices in cluster_representatives.items():  # ← assignments → cluster_representatives
    rep_text = sentences[rep_idx]
    assigned_texts = [sentences[i] for i in indices]
    all_assigned_texts.extend(assigned_texts)

    data.append({
        "대표 문장": rep_text,
        "중복 포함 할당 수": len(assigned_texts),  # 한 문장이 여러 대표에 들어가면 중복 포함
        "중복 제거 할당 수 (텍스트 기준)": len(set(assigned_texts)),
        "전체 텍스트": assigned_texts
    })

# ✅ DataFrame 생성 및 정렬 (중복 제거 기준 상위)
df_assign = pd.DataFrame(data)
df_assign_sorted = df_assign.sort_values(
    by="중복 제거 할당 수 (텍스트 기준)", ascending=False
).head(20)

# ✅ 전체 기준 할당 문장 수
total_assigned_with_duplicates = len(all_assigned_texts)
total_assigned_unique_texts = len(set(all_assigned_texts))

# ✅ 노이즈 (중복 포함 기준)
num_unassigned = total_sentences - total_assigned_with_duplicates

# ✅ 요약 정보 출력
print("\n📊 결과 요약:")
print(f"- 전체 원문 문장 수: {total_sentences}")
print(f"- 대표 문장 수: {num_representatives}")
print(f"- 대표에 할당된 문장 수 (중복 포함): {total_assigned_with_duplicates}")
print(f"- 대표에 할당된 문장 수 (텍스트 기준 중복 제거): {total_assigned_unique_texts}")
print(f"- 할당되지 않은 문장 수 (노이즈, 중복 포함 기준): {num_unassigned}")

# ✅ 📋 표 출력
pd.set_option("display.max_colwidth", None)
html_table = df_assign_sorted[[
    "대표 문장", "중복 포함 할당 수", "중복 제거 할당 수 (텍스트 기준)"
]].to_html(index=False, escape=False)

print("\n🗂️ 상위 20개 대표 문장별 할당 수 표:")
display(HTML(f"<div style='overflow-x:auto; max-width:1000px'>{html_table}</div>"))

# ✅ 예시 출력: 가장 큰 클러스터 대표 문장과 문장 출현 빈도 (상위 20개)
top_cluster = df_assign_sorted.iloc[0]
rep_text = top_cluster["대표 문장"]
rep_texts = top_cluster["전체 텍스트"]
sentence_counts = Counter(rep_texts)
top_20_sentences = dict(sentence_counts.most_common(20))

print(f"\n✅ 예시 대표 문장 1개 (가장 큰 클러스터 기준):\n대표 문장: \"{rep_text}\"")
print("클러스터 내 문장별 출현 횟수 (상위 20개):")
print(json.dumps(top_20_sentences, indent=2, ensure_ascii=False))


📊 결과 요약:
- 전체 원문 문장 수: 7406
- 대표 문장 수: 615
- 대표에 할당된 문장 수 (중복 포함): 4681
- 대표에 할당된 문장 수 (텍스트 기준 중복 제거): 2258
- 할당되지 않은 문장 수 (노이즈, 중복 포함 기준): 2725

🗂️ 상위 20개 대표 문장별 할당 수 표:



✅ 예시 대표 문장 1개 (가장 큰 클러스터 기준):
대표 문장: "elderly patients"
클러스터 내 문장별 출현 횟수 (상위 20개):
{
  "patients": 99,
  "elderly": 45,
  "aging population": 22,
  "elderly people": 22,
  "seniors": 21,
  "elderly population": 18,
  "elderly individuals": 18,
  "disabled": 17,
  "disabled population": 13,
  "senior citizens": 11,
  "disabled persons": 10,
  "employees": 9,
  "hospitals": 9,
  "elderly persons": 8,
  "healthcare providers": 7,
  "elderly patients": 7,
  "patient": 7,
  "older individuals": 7,
  "elders": 7,
  "physicians": 6
}


In [15]:
import os

# ✅ 파일 이름에서 확장자 제거하고 클러스터링 결과 이름 생성
base_name = os.path.splitext(os.path.basename(input_path))[0]
output_path = f"{base_name}_클러스터링_결과.xlsx"

# ✅ 대표문장별 통계 정리 (중복 제거 기준으로 정렬)
df_summary = df_assign[[
    "대표 문장", "중복 포함 할당 수", "중복 제거 할당 수 (텍스트 기준)"
]].sort_values(by="중복 제거 할당 수 (텍스트 기준)", ascending=False)

# ✅ 요약 텍스트 생성
summary_lines = [
    "📊 결과 요약:",
    f"- 전체 원문 문장 수: {total_sentences}",
    f"- 대표 문장 수: {num_representatives}",
    f"- 대표에 할당된 문장 수 (중복 포함): {total_assigned_with_duplicates}",
    f"- 대표에 할당된 문장 수 (텍스트 기준 중복 제거): {total_assigned_unique_texts}",
    f"- 할당되지 않은 문장 수 (노이즈, 중복 포함 기준): {num_unassigned}",
    "",
    "🗂️ 상위 대표 문장별 할당 수 표:"
]

# ✅ 시트 2: 대표문장-할당문장 구조화 (정렬된 대표 순서 기준)
rows = []
ordered_reps = df_summary["대표 문장"].tolist()

for rep in ordered_reps:
    texts = df_assign[df_assign["대표 문장"] == rep]["전체 텍스트"].values
    if len(texts) == 0:
        continue
    for txt in texts[0]:
        rows.append({"대표문장": rep, "할당된 텍스트": txt})
df_details = pd.DataFrame(rows)

# ✅ 시트 3: 할당되지 않은 문장 (알파벳 순 정렬)
all_clustered_indices = set()
for indices in cluster_representatives.values():
    all_clustered_indices.update(indices)

unassigned_texts = [sentences[i] for i in range(len(sentences)) if i not in all_clustered_indices]
df_noise = pd.DataFrame({"할당되지 않은 문장": sorted(unassigned_texts)})

# ✅ 엑셀 저장
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    # 시트 1: 요약 + 대표문장 통계
    workbook = writer.book
    worksheet1 = workbook.add_worksheet("대표문장_통계")
    writer.sheets["대표문장_통계"] = worksheet1

    # ✅ 요약 내용 작성
    for i, line in enumerate(summary_lines):
        worksheet1.write(i, 0, line)

    # ✅ 표 작성
    start_row = len(summary_lines) + 1
    for col_num, col_name in enumerate(df_summary.columns):
        worksheet1.write(start_row, col_num, col_name)
    for row_num, row_data in enumerate(df_summary.values):
        for col_num, value in enumerate(row_data):
            worksheet1.write(start_row + 1 + row_num, col_num, value)

    # 시트 2: 대표문장-할당문장
    df_details.to_excel(writer, sheet_name="대표문장_할당문장", index=False)

    # 시트 3: 노이즈 문장 (알파벳 정렬)
    df_noise.to_excel(writer, sheet_name="할당되지 않은 문장(노이즈)", index=False)

print(f"✅ 엑셀 파일 저장 완료: {output_path}")

✅ 엑셀 파일 저장 완료: FA_entities (1)_클러스터링_결과.xlsx
